In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# Inicializar a SparkSession
spark = SparkSession.builder \
    .appName("Exemplo Simples de Tabela") \
    .getOrCreate()

# Definir o schema da tabela
schema = StructType([
    StructField("id", IntegerType(), nullable=False),
    StructField("nome", StringType(), nullable=True),
    StructField("idade", IntegerType(), nullable=True),
    StructField("cidade", StringType(), nullable=True)
])

# Dados de exemplo
data = [
    (1, "João Silva", 28, "São Paulo"),
    (2, "Maria Souza", 34, "Rio de Janeiro"),
    (3, "Carlos Oliveira", 22, "Belo Horizonte"),
    (4, "Ana Santos", 45, "Curitiba")
]

# Criar DataFrame
df = spark.createDataFrame(data, schema=schema)

# Mostrar a tabela
print("Mostrando a tabela completa:")
df.show()

# Mostrar a tabela com formatação
print("\nMostrando a tabela com formatação:")
df.show(truncate=False)

# Mostrar o schema da tabela
print("\nSchema da tabela:")
df.printSchema()

Mostrando a tabela completa:
+---+---------------+-----+--------------+
| id|           nome|idade|        cidade|
+---+---------------+-----+--------------+
|  1|     João Silva|   28|     São Paulo|
|  2|    Maria Souza|   34|Rio de Janeiro|
|  3|Carlos Oliveira|   22|Belo Horizonte|
|  4|     Ana Santos|   45|      Curitiba|
+---+---------------+-----+--------------+


Mostrando a tabela com formatação:
+---+---------------+-----+--------------+
|id |nome           |idade|cidade        |
+---+---------------+-----+--------------+
|1  |João Silva     |28   |São Paulo     |
|2  |Maria Souza    |34   |Rio de Janeiro|
|3  |Carlos Oliveira|22   |Belo Horizonte|
|4  |Ana Santos     |45   |Curitiba      |
+---+---------------+-----+--------------+


Schema da tabela:
root
 |-- id: integer (nullable = false)
 |-- nome: string (nullable = true)
 |-- idade: integer (nullable = true)
 |-- cidade: string (nullable = true)

